In [ ]:
import pandas as pd


In [ ]:
website_sessions = pd.read_csv("website_sessions.csv")
orders = pd.read_csv("orders.csv")
order_items = pd.read_csv("order_items.csv")
order_item_refunds = pd.read_csv("order_item_refunds.csv")
website_pageviews = pd.read_csv("website_pageviews.csv")
products = pd.read_csv("products.csv")

In [ ]:
website_sessions["created_at"] = pd.to_datetime(website_sessions["created_at"])
orders["created_at"] = pd.to_datetime(orders["created_at"])
website_pageviews["created_at"] = pd.to_datetime(website_pageviews["created_at"])


#### Aggregate orders to SESSION level

In [ ]:
orders_per_session = (
    orders
    .groupby("website_session_id")
    .agg(
        orders=("order_id", "nunique"),
        revenue=("price_usd", "sum")
    )
    .reset_index()
)

orders_per_session.head()


##### Aggregate pageviews to SESSION level

In [ ]:
pageviews_per_session = (
    website_pageviews
    .groupby("website_session_id")
    .size()
    .reset_index(name="pageviews")
)

pageviews_per_session.head()


In [ ]:
session_level = (
    website_sessions
    .merge(orders_per_session, on="website_session_id", how="left")
    .merge(pageviews_per_session, on="website_session_id", how="left")
)

session_level[["orders", "revenue", "pageviews"]] = (
    session_level[["orders", "revenue", "pageviews"]].fillna(0)
)

session_level.head()


#### Insight:
Created a session-level dataset where each row represents one website session.

In [ ]:
session_level["is_bounce"] = session_level["pageviews"] == 1


#### Insight:
Marked sessions with only one pageview as bounced sessions.

In [ ]:
conversion_rate = (session_level["orders"] > 0).mean()
total_revenue = session_level["revenue"].sum()
avg_order_value = total_revenue / session_level["orders"].sum()
bounce_rate = session_level["is_bounce"].mean()

conversion_rate, total_revenue, avg_order_value, bounce_rate


In [ ]:
print(f"Conversion Rate: {conversion_rate:.2%}")
print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Average Order Value: ${avg_order_value:.2f}")
print(f"Bounce Rate: {bounce_rate:.2%}")


In [ ]:
channel_summary = (
    session_level
    .groupby(["utm_source","utm_campaign","device_type","utm_content"])
    .agg(
        sessions=("website_session_id", "count"),
        orders=("orders", "sum"),
        revenue=("revenue", "sum"),
        bounce_rate=("is_bounce", "mean")
    )
    .reset_index()
)

channel_summary["conversion_rate"] = (
    channel_summary["orders"] / channel_summary["sessions"]
)

channel_summary["revenue_per_session"] = (
    channel_summary["revenue"] / channel_summary["sessions"]
)

channel_summary


### Time-Based Trends (Month-wise & Year-wise)


In [ ]:
session_level["month"] = session_level["created_at"].dt.to_period("M")
session_level["year"] = session_level["created_at"].dt.year


In [ ]:
monthly_trend = (
    session_level
    .groupby("month")
    .agg(
        sessions=("website_session_id", "count"),
        orders=("orders", "sum"),
        revenue=("revenue", "sum")
    )
    .reset_index()
)

monthly_trend


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(monthly_trend["month"].astype(str), monthly_trend["sessions"])
plt.xticks(rotation=45)
plt.title("Monthly Sessions Trend")
plt.xlabel("Month")
plt.ylabel("Sessions")
plt.tight_layout()
plt.show()


In [ ]:
yearly_trend = (
    session_level
    .groupby("year")
    .agg(
        sessions=("website_session_id", "count"),
        orders=("orders", "sum"),
        revenue=("revenue", "sum")
    )
    .reset_index()
)

yearly_trend


In [ ]:
plt.figure()
plt.plot(yearly_trend["year"], yearly_trend["revenue"])
plt.title("Yearly Revenue Trend")
plt.xlabel("Year")
plt.ylabel("Revenue")
plt.show()


In [ ]:
len(orders), len(orders_per_session)


In [ ]:
len(website_pageviews), len(pageviews_per_session)


In [ ]:
session_level["website_session_id"].is_unique


## Advanced Visualizations

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


In [ ]:
fig = px.bar(
    channel_summary,
    x="utm_source",
    y=["sessions", "orders"],
    barmode="group",
    title="Traffic vs Orders by Marketing Channel"
)

fig.show()


In [ ]:
fig = px.bar(
    channel_summary,
    x="utm_source",
    y="conversion_rate",
    text_auto=".2%",
    title="Conversion Rate by Marketing Channel"
)

fig.update_yaxes(tickformat=".0%")
fig.show()


In [ ]:
fig = px.bar(
    channel_summary,
    x="utm_source",
    y="revenue_per_session",
    text_auto=".2f",
    title="Revenue per Session by Marketing Channel"
)

fig.show()


In [ ]:
fig = px.scatter(
    channel_summary,
    x="sessions",
    y="conversion_rate",
    size="revenue",
    color="utm_source",
    title="Traffic vs Conversion Rate by Channel",
    labels={
        "sessions": "Total Sessions",
        "conversion_rate": "Conversion Rate"
    }
)

fig.update_yaxes(tickformat=".0%")
fig.show()


In [ ]:
fig = px.scatter(
    channel_summary,
    x="bounce_rate",
    y="conversion_rate",
    size="sessions",
    color="utm_source",
    title="Bounce Rate vs Conversion Rate by Channel",
    labels={
        "bounce_rate": "Bounce Rate",
        "conversion_rate": "Conversion Rate"
    }
)

fig.update_xaxes(tickformat=".0%")
fig.update_yaxes(tickformat=".0%")
fig.show()


In [ ]:
monthly_trend["month"] = monthly_trend["month"].dt.to_timestamp()


In [ ]:
fig = px.line(
    monthly_trend,
    x="month",
    y="sessions",
    title="Monthly Sessions Trend"
)

fig.show()


In [ ]:
fig = px.line(
    monthly_trend,
    x="month",
    y="revenue",
    title="Monthly Revenue Trend"
)

fig.show()


In [ ]:
session_level["month"] = session_level["month"].dt.to_timestamp()


In [ ]:
monthly_channel = (
    session_level
    .groupby(["month", "utm_source"])
    .agg(sessions=("website_session_id", "count"))
    .reset_index()
)


In [ ]:
fig = px.line(
    monthly_channel,
    x="month",
    y="sessions",
    color="utm_source",
    title="Monthly Sessions Trend by Channel"
)

fig.show()


In [ ]:
device_channel = (
    session_level
    .groupby(["utm_source", "device_type"])
    .agg(
        sessions=("website_session_id", "count"),
        orders=("orders", "sum")
    )
    .reset_index()
)

device_channel["conversion_rate"] = (
    device_channel["orders"] / device_channel["sessions"]
)


In [ ]:
fig = px.bar(
    device_channel,
    x="utm_source",
    y="conversion_rate",
    color="device_type",
    barmode="group",
    title="Conversion Rate by Channel and Device"
)

fig.update_yaxes(tickformat=".0%")
fig.show()


In [ ]:
fig = px.pie(
    channel_summary,
    names="utm_source",
    values="revenue",
    hole=0.4,
    title="Revenue Contribution by Marketing Channel"
)

fig.show()


# Final Visualization

## 4 KPIs

In [ ]:
total_sessions = session_level["website_session_id"].count()
bounce_rate = session_level["is_bounce"].mean()
conversion_rate = (session_level["orders"] > 0).mean()
avg_revenue_per_session = session_level["revenue"].mean()

total_sessions, bounce_rate, conversion_rate, avg_revenue_per_session


In [ ]:
print(f"Total Sessions: {total_sessions:,}")
print(f"Bounce Rate: {bounce_rate:.2%}")
print(f"Conversion Rate: {conversion_rate:.2%}")
print(f"Avg Revenue per Session: ${avg_revenue_per_session:.2f}")


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode="number",
    value=total_sessions,
    title={"text": "Total Sessions"},
    domain={'row': 0, 'column': 0}
))

fig.add_trace(go.Indicator(
    mode="number",
    value=bounce_rate * 100,
    number={'suffix': "%"},
    title={"text": "Bounce Rate"},
    domain={'row': 0, 'column': 1}
))

fig.add_trace(go.Indicator(
    mode="number",
    value=conversion_rate * 100,
    number={'suffix': "%"},
    title={"text": "Conversion Rate"},
    domain={'row': 1, 'column': 0}
))

fig.add_trace(go.Indicator(
    mode="number",
    value=avg_revenue_per_session,
    number={'prefix': "$"},
    title={"text": "Avg Revenue / Session"},
    domain={'row': 1, 'column': 1}
))

fig.update_layout(
    grid={'rows': 2, 'columns': 2},
    title="Marketing Performance KPIs"
)

fig.show()


## Traffic vs Orders

In [ ]:
monthly_summary = (
    session_level
    .groupby("month")
    .agg(
        sessions=("website_session_id", "count"),
        orders=("orders", "sum")
    )
    .reset_index()
)


In [ ]:
fig = px.line(
    monthly_summary,
    x="month",
    y=["sessions", "orders"],
    title="Traffic vs Orders Over Time"
)

fig.show()


## Traffic vs Conversion Rate

In [ ]:
fig = px.scatter(
    channel_summary,
    x="sessions",
    y="conversion_rate",
    size="revenue",
    color="utm_source",
    title="Traffic vs Conversion Rate"
)

fig.update_yaxes(tickformat=".0%")
fig.show()


## Revenue Pie

In [ ]:
fig = px.pie(
    channel_summary,
    names="utm_source",
    values="revenue",
    hole=0.4,
    title="Revenue Distribution by Channel"
)

fig.show()


## Traffic Pie

In [ ]:
fig = px.pie(
    channel_summary,
    names="utm_source",
    values="sessions",
    hole=0.4,
    title="Traffic Distribution by Channel"
)

fig.show()


## Campaign Performance

In [ ]:
campaign_summary = (
    session_level
    .groupby(["utm_campaign", "utm_source"])
    .agg(
        sessions=("website_session_id", "count"),
        orders=("orders", "sum")
    )
    .reset_index()
)

campaign_summary["conversion_rate"] = (
    campaign_summary["orders"] / campaign_summary["sessions"]
)


In [ ]:
fig = px.bar(
    campaign_summary,
    y="utm_campaign",
    x="sessions",
    color="utm_source",
    orientation="h",
    title="Campaign Performance by Traffic"
)

fig.show()
